In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt 
import io
import sys
import os

In [2]:
EPOCH = 10
BATCH_SIZE = 32
LR = 0.0001

In [3]:
T = transforms.ToTensor()
train_data = datasets.CIFAR10(root='~/newStuFile/ZHX/dataset/', train=True,transform=T,download=True)
test_data =datasets.CIFAR10(root='~/newStuFile/ZHX/dataset/',train=False,transform=T,download=True)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

In [5]:
sys.path.append('..')
from alexnet import alexnet_cifar
from vgg_19 import vgg19_cifar
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset=train_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)
test_loader = DataLoader(dataset=test_data,batch_size=BATCH_SIZE,shuffle=True,num_workers=2)
model = vgg19_cifar().cuda()

In [6]:
len(train_loader)

1563

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=LR)

In [ ]:
# model = torch.load('../pts/cifar_alexnet.pt')

In [8]:
model.train()
from tqdm import tqdm
with tqdm(total = EPOCH*len(train_loader)) as pbar:
    for epoch in range(EPOCH):
        for i,data in enumerate(train_loader):
            inputs,labels = data
            inputs,labels = inputs.cuda(),labels.cuda()
            outputs = model(inputs)
            loss = criterion(outputs,labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            pbar.update(1)
            pbar.desc = '%f'%loss.item()

0.006962: 100%|██████████| 15630/15630 [2:33:16<00:00,  1.70it/s] 


In [9]:
torch.save(model,'../pts/cifar_vgg_19.pt')

In [10]:
model.eval()
correct,total = 0,0
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=LR)
with tqdm(total = len(test_loader)) as pbar:
    for j,data in enumerate(test_loader):
        inputs,labels = data
        inputs,labels = inputs.cuda(),labels.cuda()
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data,1)
        total =total+labels.size(0)
        correct = correct +(predicted == labels).sum().item()
        pbar.update(1)
print('Accuracy：{:.4f}%'.format(100.0*correct/total))

100%|██████████| 313/313 [00:59<00:00,  5.29it/s]

Accuracy：80.1000%


In [12]:
def train(model, train_loader, save_name) -> nn.Module:
    EPOCH = 10
    BATCH_SIZE = 32
    LR = 0.0001
    model = model.cuda()
    model.train()
    with tqdm(total = EPOCH*len(train_loader)) as pbar:
        for epoch in range(EPOCH):
            for i,data in enumerate(train_loader):
                inputs,labels = data
                inputs,labels = inputs.cuda(),labels.cuda()
                outputs = model(inputs)
                loss = criterion(outputs,labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                pbar.update(1)
                pbar.desc = '%f'%loss.item()
    torch.save(model,'../pts/%s'%save_name)
    return model

In [13]:
from resnet_50 import resnet50_cifar
resnet50_model = resnet50_cifar()
train(resnet50_model,train_loader,'cifar_resnet_50')

2.352819:   0%|          | 29/15630 [00:11<1:46:06,  2.45it/s]